![](./img/1.png)

![](./img/2.png)

1. 用户输入问题，进入 agent 分诊节点进行用户意图分析  
2. 若需调用工具，则调用 call.tools 节点进行工具并行执行；否则直接回复用户问题结束  
3. 工具调用后，根据工具类型进行路由：

- 检索类工具: 进入 grade Documents 进行相关性评分, 若相关则路由到 generate 节点, 否则 rewrite (最多3次)  
。非检索类工具：直接路由到 generate 节点

4. 生成最终回复并输出


# 1.2核心功能

# (1) 核心功能：基于状态图的对话流程

定义了一个包含多个节点 (Node) 和边 (Edge) 的对话工作流, 节点包括:

- agent: 分析用户问题并决定是否调用工具  
- call.tools: 并行执行工具调用  
- grade_documents: 评估检索到的文档与问题的相关性  
- rewrite: 重写用户查询以改进问题（查询意图识别查询改写）  
- generate: 生成最终回复  
- 动态路由：

- tools_condition: 根据用户查询意图识别是否需要调用工具  
- route_after.tools：根据工具调用结果决定下一步是生成回复还是评分文档  
- route_after_grade: 根据文档相关性评分决定生成回复还是重写查询

# (2)工具调用与并行处理

- 工具配置：通过 ToolConfig 类管理工具列表和路由配置，支持动态路由（检索类工具路由到grade_documents，其他路由到 generate）  
- 并行工具节点：ParallelToolNode 使用线程池（ThreadPoolExecutor）并行执行多个工具调用，提高效率，支持最大工作线程数配置（默认5）


# (3) 数据库与持久化存储

- 连接池管理：使用ConnectionPool管理PostgreSQL数据库连接，支持自动提交、超时设置和连接池状态监控（monitor_connection_pool）  
- 线程内持久化：通过PostgresSaver保存对话状态检查点  
- 跨线程持久化：通过PostgresStore存储用户记忆（store_memory），支持记忆搜索和存储（如用户输入包含“记住”时保存）  
- 重试机制：数据库操作使用tenacity库实现重试（最多3次，指数退避等待），提高健壮性

# （4）自然语言处理与提示模板

- 语言模型：通过get_Idm获取聊天模型（llm chatting）和嵌入模型（llm_embedding），用于处理用户输入和生成回复  
- 提示模板：使用create_chain函数加载并缓存提示模板文件（如代理、重写、评分、生成模板），支持结构化输出（如DocumentRelevanceScore）  
- 消息过滤：filter_messages过滤消息，仅保留AIMessage和HumanMessage，限制最多保留5条历史消息

# (5) 日志与错误处理

- 日志记录：使用logging模块和ConcurrentRotatingFileHandler记录详细日志（DEBUG级别），支持文件轮转（5MB，3个备份），后续可以自己实现日志归档  
- 错误处理：多层次异常捕获（如数据库连接、工具调用、状态访问等），记录错误并提供默认路由或提示用户

# (6) 用户交互与响应输出

- 主循环：main 函数实现交互式对话，用户输入后通过graph_response处理并输出响应，支持退出命令（quit、exit、q）  
- 响应区分：区分工具输出（显示工具名称）和大模型输出（普通回复），提升用户体验

# (7) 辅助功能

可视化：通过save_graphVISUALIZATION将状态图保存为PNG文件，便于调试

配置管理：通过Config类统一管理日志文件路径、数据库URI、提示模板路径等配置

# 2. 项目环境准备

## 2.1 创建conda环境

# 2.3 使用Docker运行PostgreSQL数据库

1）进入官网https://www.docker.com/下载安装Docker Desktop软件并安装，安装完成后打开软件

Docker Desktop 安装参考：https://www.cnblogs.com/yinzhengjie/p/17889373.html

2）打开命令行终端，找到PostgreSQL的docker配置文件docker-compose.yml  
3）运行 docker-compose up -d 命令后台启动 PostgreSQL 数据库服务。运行成功后可在Docker Desktop软件中进行管理操作或使用命令行操作或使用指令

若要删除容器在命令行终端中运行 docker-compose down --volumes

4）因为LangGraph中PostgresStore需要使用到pgvector，因此需要在docker容器中按照如下步骤进行操作，直接使用Docker Desktop软件中进行操作

1. 安装依赖

apt update

apt install -y git build-essential postgrepql-server-dev-15

2. 编译并安装 pgvector

git clone --branch v0.7.0 https://github.com/pgvector/pgvector.git

cd pgvector

make

make install
